In [ ]:
#@title install dependencies
!git clone https://github.com/ashawkey/RAD-NeRF.git

%cd RAD-NeRF

%mkdir -p pretrained
%mkdir -p data

Cloning into 'RAD-NeRF'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 124 (delta 21), reused 12 (delta 12), pack-reused 89
Receiving objects: 100% (124/124), 111.88 KiB | 15.98 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/RAD-NeRF


### Download the model and Setting up Thin-Plate-Spline-Motion-Model for Generating Talking Avatar

In [6]:
!gdown --id 1MB7jE9pl-k9aWd_TY3IEeCe9T3wfHOgT
!unzip TPSMM.zip

!gdown --id 1-CKOjv_y_TzNe-dwQsjjeVxJUuyBAb5X --output TPSMM/vox.pth.tar

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1MB7jE9pl-k9aWd_TY3IEeCe9T3wfHOgT
To: /content/TPSMM.zip
100% 67.1M/67.1M [00:00<00:00, 115MB/s]


### Installing all the dependencies

In [ ]:
# !pip install streamlit
# !pip install -U elevenlabs
# !pip install face_alignment
# !pip install imageio_ffmpeg
# !pip3 install wldhx.yadisk-direct
# !pip install pyngrok openai
# !pip install git+https://github.com/openai/whisper.git

# !pip install gradio
# !pip install streamlit-audiorecorder

### Installing all the Libraries and configuration for the RAD-NeRF model

In [ ]:
# install (slow...)
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install -r requirements.txt

!pip install ffmpeg-python
!bash scripts/install_ext.sh

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 2s (108 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 120831 files and directories currently installed.)
Pre

### Start working on Model

In [ ]:
%cd /content/RAD-NeRF

/content/RAD-NeRF


In [ ]:
Person = 'engm' #@param ['obama', 'marco', 'engm', 'chris']
Audio = 'custom' #@param ['intro', 'nvp', 'custom']
Background = 'default' #@param ['default', 'custom']
Pose_start = 0 #@param {type: 'integer'}
Pose_end = 100 #@param {type: 'integer'}


In [ ]:
#@title Download things for RAD-NeRF Repo

import gdown

# model checkpoint and pose sequence
if Person == 'obama':
    gdown.download("https://drive.google.com/uc?id=1dlajKAkibAWRzNWaUdoFO0OKhVeeBW9w", "pretrained/model.pth", quiet=False)
    gdown.download("https://drive.google.com/uc?id=1_PK1AZCpB7CCBiQsvOm14qr31K20TV9J", "data/pose.json", quiet=False)
elif Person == 'engm':
    gdown.download("https://drive.google.com/uc?id=1P4Fbg5IN_eWxkr7sVQJeK95WVnzV0yGs", "pretrained/model.pth", quiet=False)
    gdown.download("https://drive.google.com/uc?id=1bmkohn1bnxkCJs9Igv5xXIBhyWKoyXMb", "data/pose.json", quiet=False)
elif Person == 'marco':
    gdown.download("https://drive.google.com/uc?id=1XkLbWF3CbvIaLTkjUd7zkdp03um1cyPL", "pretrained/model.pth", quiet=False)
    gdown.download("https://drive.google.com/uc?id=1ETV7yyK3u4MC1ubUQbIvTrRAfJvypcUg", "data/pose.json", quiet=False)
elif Person == 'chris':
    gdown.download("https://drive.google.com/uc?id=1Mx2f1Iu-W9yG8QOn_2ROhhIcrVz4yMeA", "pretrained/model.pth", quiet=False)
    gdown.download("https://drive.google.com/uc?id=1O8uvNuqMo4p0aJi16OMJXLsd1UeQ_7Dw", "data/pose.json", quiet=False)


# audio example
if Audio == 'intro':
    gdown.download("https://drive.google.com/uc?id=1gxfLuYuGFMlYz7BDxJlcfnDISN4sfbKg", "data/intro.wav", quiet=False)
    gdown.download("https://drive.google.com/uc?id=1r2-wJ0VFfahE4bdDzGgvoC2tU24XmxPu", "data/intro_eo.npy", quiet=False)
elif Audio == 'nvp':
    gdown.download("https://drive.google.com/uc?id=1WgtSiB_gLZZ-b8IO_KIVwT_ZRPUSx7yY", "data/nvp.wav", quiet=False)
    gdown.download("https://drive.google.com/uc?id=1UKCW67Idw00_3QqzXasN_dvYXlYC37fe", "data/nvp_eo.npy", quiet=False)


# background image
if Background == 'default':
    gdown.download("https://drive.google.com/uc?id=1RDiRCFXMk3hQ0hzKVculQdFqisayT1MM", "data/bg.jpg", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1P4Fbg5IN_eWxkr7sVQJeK95WVnzV0yGs
To: /content/RAD-NeRF/pretrained/model.pth
100%|██████████| 17.3M/17.3M [00:00<00:00, 31.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bmkohn1bnxkCJs9Igv5xXIBhyWKoyXMb
To: /content/RAD-NeRF/data/pose.json
100%|██████████| 2.08M/2.08M [00:00<00:00, 237MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RDiRCFXMk3hQ0hzKVculQdFqisayT1MM
To: /content/RAD-NeRF/data/bg.jpg
100%|██████████| 51.3k/51.3k [00:00<00:00, 48.6MB/s]


In [ ]:
%cd /content/RAD-NeRF

/content/RAD-NeRF


#### Generating Talking Avatar Template from RAD-NeRF

In [ ]:
#markdown ####**Settings:**

import os, whisper
import torchaudio
import datetime
import gradio as gr
from subprocess import run
from audio_avatar import generate_audio

def avatar_generation(audio):
    Person = 'engm' #param ['obama', 'marco', 'engm', 'chris']
    Audio = 'custom' #param ['intro', 'nvp', 'custom']
    Background = 'default' #param ['default', 'custom']
    Pose_start = 0 #param {type: 'integer'}
    Pose_end = 100 #param {type: 'integer'}

    parent = 'data'
    now = datetime.datetime.now()
    time_ = now.time()
    time_var = time_.isoformat().split('.')[0]
    time_var = time_var.replace(':', '_')
    audio_name = parent + '/' + time_var + '.wav'


    audio_read, sr = torchaudio.load(audio)
    torchaudio.save(audio_name, audio_read, sample_rate=sr)
    model = whisper.load_model("base")
    result = model.transcribe(audio_name)
    user_prompt = result["text"]
    chat_Aud = generate_audio(user_prompt)
    # if Audio == 'custom':
    print('Getting AI features...', '\n')
    run(['python', 'nerf/asr.py', '--wav', chat_Aud, '--save_feats'])

    BG = 'bg.jpg'
    audio_npy = 'data/' + chat_Aud.split('data/')[1][:-4] + '_eo.npy'
    # audio_npy = '/content/RAD-NeRF/data/chatgpt_response_20_25_59_eo.npy'
    bg_img = 'data/' + BG
    print('Generating Avatar')

    os.system(f"python test.py -O --torso --pose data/pose.json --data_range {Pose_start} {Pose_end} --ckpt pretrained/model.pth --aud {audio_npy} --bg_img {bg_img} --workspace trail")

    Video = 'trail/results/ngp_ep0028.mp4'
    Video_aud = Video.replace('.mp4', '_aud.mp4')

    # concat audio
    os.system(f"ffmpeg -y -i {Video} -i {chat_Aud} -c:v copy -c:a aac {Video_aud}")
    return Video

### Generate talking avatar for the target image, with audio based generated video template

In [ ]:
import torch, cv2
import streamlit as st
import time, os, shutil, datetime
import imageio
from moviepy.editor import *
from PIL import Image
import imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
from TPSMM.demo import make_animation
from skimage import img_as_ubyte
# from rad_nerf import avatar_generation

warnings.filterwarnings("ignore")

from TPSMM.demo import load_checkpoints
from audio_avatar import generate_audio
import streamlit as st
import openai, whisper
from elevenlabs import generate, play, save, set_api_key

set_api_key('')
openai.api_key = ""

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):

    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def save_uploadedfile(uploadedfile):
    os.makedirs('tempDir', exist_ok=True)
    video_file_name = os.path.join("tempDir", uploadedfile.name)
    with open(video_file_name, "wb") as f:
        f.write(uploadedfile.getbuffer())

    return video_file_name


def generate_animation(image_file, voice_file):
    parent = 'data'
    now = datetime.datetime.now()
    time_ = now.time()
    time_var = time_.isoformat().split('.')[0]
    time_var = time_var.replace(':', '_')
    audio_name = parent + '/' + time_var + '.wav'


    audio_read, sr = torchaudio.load(voice_file)
    torchaudio.save(audio_name, audio_read, sample_rate=sr)
    # edit the config
    device = torch.device('cuda:0')
    dataset_name = 'vox'  # ['vox', 'taichi', 'ted', 'mgif']
    # source_image_path = image
    # driving_video_path = driving_video_1

    # image_uploaded = save_uploadedfile(image_file)

    try:
        voice_file_name = save_uploadedfile(voice_file)
    except:
        voice_file_name = audio_name
    print('Performing voice commands with ChatGPT response...')
    model = whisper.load_model("base")
    result = model.transcribe(voice_file_name)
    voice_prompt = result["text"]

    print("""Generating Voice ChatGPT Response...""")
    chatgpt_response = get_completion(voice_prompt)
    audio_response = generate_audio(chatgpt_response)

    parent = '/content'
    now = datetime.datetime.now()
    time_ = now.time()
    time_var = time_.isoformat().split('.')[0]
    time_var = time_var.replace(':', '_')

    output_video_path = parent + '/' + time_var + '_animation' + '.mp4'

    config_path = 'TPSMM/config/vox-256.yaml'
    checkpoint_path = 'TPSMM/vox.pth.tar'

    predict_mode = 'relative'  # ['standard', 'relative', 'avd']
    find_best_frame = False  # when use the relative mode to animate a face, use 'find_best_frame=True' can get better quality result

    pixel = 256  # for vox, taichi and mgif, the resolution is 256*256
    if (dataset_name == 'ted'):  # for ted, the resolution is 384*384
        pixel = 384

    # source_image = imageio.imread(image_uploaded)

    print('Rendering video animation...', '\n')
    ### driving video rendering

    os.makedirs('tempDir', exist_ok=True)

    """
    Here we will generate talking avatar template based on our audio.
    """

    talking_person = avatar_generation(audio_response)

    reader = imageio.get_reader(talking_person)

    source_image = resize(image_file, (pixel, pixel))[..., :3]

    fps = reader.get_meta_data()['fps']
    render_video = []
    try:
        for im in reader:
            render_video.append(im)
    except RuntimeError:
        pass
    reader.close()

    driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in render_video]

    inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path=config_path,
                                                                                checkpoint_path=checkpoint_path,
                                                                                device=device)

    if predict_mode == 'relative' and find_best_frame:
        from TPSMM.demo import find_best_frame as _find
        i = _find(source_image, driving_video, device.type == 'cpu')
        print("Best frame: " + str(i))
        driving_forward = driving_video[i:]
        driving_backward = driving_video[:(i + 1)][::-1]
        predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector,
                                            dense_motion_network, avd_network, device=device, mode=predict_mode)
        predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector,
                                            dense_motion_network, avd_network, device=device, mode=predict_mode)
        predictions = predictions_backward[::-1] + predictions_forward[1:]
    else:

        print('Converting image to Digital Replica')
        predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network,
                                    avd_network, device=device, mode=predict_mode)

    # save resulting video
    print("""Saving animated video...""")
    imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

    # audio_file = text_to_speech(chatgpt_response)

    videoclip = VideoFileClip(output_video_path)
    audioclip = AudioFileClip(audio_response)

    new_audioclip = CompositeAudioClip([audioclip])
    videoclip.audio = new_audioclip

    audio_video_file = 'audio_video.mp4'
    videoclip.write_videofile(audio_video_file)

    return audio_video_file



In [ ]:
gr.Interface(
    title = """
    🗣️🔄💬 Communication with Cloneality Voice Cloner 🔄💬🗣️""",

    description = """
    Welcome to Cloneality Voice Cloner, your gateway to next-level communication! 🚀🎤 Our innovative website combines the power of ChatGPT and ElevenLabs API, offering an unparalleled experience in voice-to-voice communication. 🤝🎙️

    Simply engage with ChatGPT, and witness its magic as it clones your voice, creating a lifelike audio representation. 🎉🔊 Give voice instructions, share thoughts, and receive responses - all in your own voice! 🎯🗣️

    With Voice-Cloner Connect, the possibilities are endless! Embrace the future of interactive and personalized conversations. 🌐🔮 Elevate your interactions today and unlock a new realm of communication! 🌟💬

    """,
    fn=generate_animation,
    inputs=[
        gr.Image(label='Target Image'),
        gr.Audio(source="microphone", type="filepath",)

        #streaming=True)
    ],
    outputs=[
        gr.Video(label="Avatar Response")
    ]).launch(debug=True, share=True)

In [ ]:
#@title Display Video

import os
import glob
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width=450):

  video_file = open(video_path, "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"

  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

show_video(Video_aud)
